In [1]:
import pandas as pd
from scipy import sparse as sp
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

In [2]:
import pandas as pd
from scipy import sparse as sp
import pandas as pd
import numpy as np
import torch

import torch.nn as nn
#ppi_matrix=sp.load_npz('./data/ppi_matrix7132.npz')
all_sequence_outputsnew=np.load('./data/all_sequence_outputs7132.npy')
merged_df=pd.read_csv('./data/merged_df7132.csv')

一个蛋白对于多个transcript，这部分代码是基于蛋白的ppi

In [6]:
#ppi直接加载构建好的npz，以下是构建npz步骤
ppi = pd.read_csv('./data/ppi3ensp.csv')
print(ppi.protein_id2.unique().shape)
gene_id_dict = {gene: i for i, gene in enumerate(merged_df['protein'])}
#将ppi中的Protein1变为protein_id1在gene_id_dict中对应的序号
ppi['Protein1'] = ppi['protein_id1'].map(gene_id_dict)
#将ppi中的Protein2变为protein_id2在gene_id_dict中对应的序号
ppi['Protein2'] = ppi['protein_id2'].map(gene_id_dict)
ppi=ppi.dropna()
#用ppi构建一个coo的稀疏举证，用protein1和protein2构建一个coo的稀疏矩阵
ppi_matrix = sp.coo_matrix((ppi["CombinedScore"], (ppi['Protein1'], ppi['Protein2'])), shape=(merged_df.shape[0], merged_df.shape[0]))

ppi_matrix

(41441,)


<7132x7132 sparse matrix of type '<class 'numpy.int64'>'
	with 364726 stored elements in COOrdinate format>

In [7]:
adj=ppi_matrix.todense()
adj.shape

(7132, 7132)

加图之前

In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from src.utils import set_seed
from sklearn.metrics import r2_score
import torch.nn.functional as F

# 设置随机种子
SEED = 5
set_seed(SEED)

# 确保 merged_df 和 all_sequence_outputsnew 已定义
X = torch.tensor(np.array(merged_df['rNC2'], dtype=np.float32).reshape(-1, 1))
y = torch.tensor(np.array(merged_df['NC3'], dtype=np.float32).reshape(-1, 1))
paired_ratio = torch.tensor(np.array(merged_df['High_Pause_Counts'], dtype=np.float32).reshape(-1, 1))
sequence_embedding = torch.tensor(all_sequence_outputsnew, dtype=torch.float32)

# 检查 sequence_embedding 的形状
print(f'sequence_embedding shape: {sequence_embedding.shape}')

# 假设 sequence_embedding 的第二个维度应该是 9126，根据错误信息进行修正
# 如果实际的 shape 不是 9126，请根据实际情况进行调整
if sequence_embedding.shape[1] != 9126:
    print(f'Expected sequence_embedding shape to be (N, 9126), but got {sequence_embedding.shape}')

# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=SEED)
sequence_embedding_train, sequence_embedding_test = train_test_split(sequence_embedding, test_size=0.1, random_state=SEED)
paired_ratio_train, paired_ratio_test = train_test_split(paired_ratio, test_size=0.1, random_state=SEED)

# 创建数据加载器
train_dataset = TensorDataset(X_train, sequence_embedding_train, paired_ratio_train, y_train)
test_dataset = TensorDataset(X_test, sequence_embedding_test, paired_ratio_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc_x = nn.Sequential(
            nn.Linear(1, 32),
            nn.GELU()
        )
        
        self.fc_paired = nn.Sequential(
            nn.Linear(1, 32),
            nn.GELU()
        )
        
        self.encoder = nn.Sequential(
            nn.Linear(9216, 32),
            nn.GELU()
        )

        self.fc1 = nn.Sequential(
            nn.Linear(64, 32),
            nn.GELU(),
            nn.Linear(32, 1)
        )

    def forward(self, x, sequence_embedding, paired_ratio):
        x_out = self.fc_x(x)
        paired_out = self.fc_paired(paired_ratio)
        seq_out = self.encoder(sequence_embedding)
        
        x = x_out  + seq_out
        x =  torch.cat((paired_out,x), dim=1)
        x = self.fc1(x)
        #print(x.shape)
        return x
    

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
neural_net = NeuralNet().to(device)

# 损失函数和优化器
optimizer = optim.Adam(neural_net.parameters(), lr=3e-4)
criterion = nn.MSELoss()

num_epochs = 100000  # 为了示例，使用更少的epochs

for epoch in range(num_epochs):
    neural_net.train()
    total_loss_epoch = 0

    for x, seq_emb, pair_emb, y in train_loader:  # 使用批量数据

        x = x.to(device)
        seq_emb = seq_emb.to(device)
        pair_emb = pair_emb.to(device)
        y = y.to(device)

        y_pred = neural_net(x, seq_emb, pair_emb)

        loss = criterion(y, y_pred)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss_epoch += loss.item()

    if epoch % 1 == 0:
        neural_net.eval()  # Set the neural_net to evaluation mode
        with torch.no_grad():  # No need to track gradients for validation
            X_test = X_test.to(device)
            sequence_embedding_test = sequence_embedding_test.to(device)
            paired_ratio_test = paired_ratio_test.to(device)
            
            y_pred = neural_net(X_test, sequence_embedding_test, paired_ratio_test).cpu().numpy()
            r2 = r2_score(y_test, y_pred)
        print(f'Total Loss: {total_loss_epoch/len(train_loader)}, Epoch: {epoch}, R²: {r2:.4f}')


seed = 5
sequence_embedding shape: torch.Size([7132, 9216])
Expected sequence_embedding shape to be (N, 9126), but got torch.Size([7132, 9216])
Total Loss: 2063767.635055693, Epoch: 0, R²: 0.0285
Total Loss: 1392380.1432549504, Epoch: 1, R²: 0.2854
Total Loss: 1205721.994740099, Epoch: 2, R²: 0.3584
Total Loss: 1138004.0949876239, Epoch: 3, R²: 0.4019
Total Loss: 1111101.6893564356, Epoch: 4, R²: 0.4267
Total Loss: 1099185.6492883663, Epoch: 5, R²: 0.4356
Total Loss: 1091923.3106435644, Epoch: 6, R²: 0.4442
Total Loss: 1107293.6551670793, Epoch: 7, R²: 0.4454
Total Loss: 1195094.6649133663, Epoch: 8, R²: 0.4481
Total Loss: 1088374.1629022278, Epoch: 9, R²: 0.4497
Total Loss: 1083429.2586633663, Epoch: 10, R²: 0.4519
Total Loss: 1081404.3058477722, Epoch: 11, R²: 0.4486
Total Loss: 1084103.0238242573, Epoch: 12, R²: 0.4549
Total Loss: 1116282.0287747525, Epoch: 13, R²: 0.4552
Total Loss: 1090327.609839109, Epoch: 14, R²: 0.4545
Total Loss: 1077519.6036509902, Epoch: 15, R²: 0.4585
Total

KeyboardInterrupt: 

加图

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv
from sklearn.metrics import r2_score
import torch_geometric.transforms as T
import numpy as np
import pandas as pd
from src.utils import set_seed

# 设置随机种子
SEED = 5
set_seed(SEED)

# 假定 featureDF, ppi_matrix, 和 all_sequence_outputsnew 已经定义好

# 转换为PyTorch Tensor
edge_index = torch.tensor(np.array([ppi_matrix.row, ppi_matrix.col]), dtype=torch.long)

# 创建PyTorch Geometric的Data对象
graph = Data(
    x=torch.tensor(merged_df['rNC2'].to_numpy(), dtype=torch.float32).unsqueeze(1),
    edge_index=edge_index,
    y=torch.tensor(merged_df['NC3'].to_numpy(), dtype=torch.float32).unsqueeze(1),
    seq=torch.tensor(all_sequence_outputsnew, dtype=torch.float32),
    pause=torch.tensor(merged_df['High_Pause_Counts'].to_numpy(), dtype=torch.float32).unsqueeze(1)
)

# 划分训练集和测试集
split = T.RandomNodeSplit(num_test=0.1)
graph = split(graph)

class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(1, 32)
        )

        self.encoder = nn.Sequential(
            nn.Linear(9216, 32),
            nn.GELU(),
        )

        self.fc1 = nn.Sequential(
            nn.Linear(64, 32),
            nn.GELU(),
            nn.Linear(32, 32),
        )


        self.conv1 = SAGEConv(32, 1)

    def forward(self, data):
        x = data.x
        seq_embedding = data.seq
        edge_index = data.edge_index
        pausescore = data.pause

        x = self.fc(x) + self.encoder(seq_embedding)
        x = torch.cat((self.fc(pausescore), x), dim=1)
        x = F.gelu(x)
        x = self.fc1(x)
        x = self.conv1(x, edge_index)

        return x

device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
neural_net = NeuralNet().to(device)

# 损失函数和优化器
optimizer = optim.Adam(neural_net.parameters(), lr=0.001)
criterion = nn.MSELoss()

num_epochs = 700

graph = graph.to(device)

for epoch in range(1, num_epochs + 1):
    neural_net.train()
    optimizer.zero_grad()

    # 获取原始特征和掩码特征
    out = neural_net(graph)
    train_loss = criterion(out[graph.train_mask], graph.y[graph.train_mask])
    train_loss.backward()
    optimizer.step()

    if epoch % 1 == 0:
        neural_net.eval()
        with torch.no_grad():
            out = neural_net(graph)
            train_loss = criterion(out[graph.train_mask], graph.y[graph.train_mask])
            train_r2 = r2_score(graph.y[graph.train_mask].cpu().numpy(), out[graph.train_mask].cpu().numpy())

            test_loss = criterion(out[graph.test_mask], graph.y[graph.test_mask])
            test_r2 = r2_score(graph.y[graph.test_mask].cpu().numpy(), out[graph.test_mask].cpu().numpy())

        print(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.3f}, Train R²: {train_r2:.3f}, Test Loss: {test_loss:.3f}, Test R²: {test_r2:.3f}')


seed = 5
Epoch: 001, Train Loss: 2342620.750, Train R²: -0.357, Test Loss: 2282977.000, Test R²: -0.379
Epoch: 002, Train Loss: 2328652.750, Train R²: -0.349, Test Loss: 2269429.500, Test R²: -0.371
Epoch: 003, Train Loss: 2314751.750, Train R²: -0.341, Test Loss: 2255956.000, Test R²: -0.363
Epoch: 004, Train Loss: 2300677.500, Train R²: -0.333, Test Loss: 2242305.750, Test R²: -0.355
Epoch: 005, Train Loss: 2286431.250, Train R²: -0.325, Test Loss: 2228492.000, Test R²: -0.346
Epoch: 006, Train Loss: 2271661.750, Train R²: -0.316, Test Loss: 2214159.750, Test R²: -0.338
Epoch: 007, Train Loss: 2256204.000, Train R²: -0.307, Test Loss: 2199139.500, Test R²: -0.329
Epoch: 008, Train Loss: 2239956.750, Train R²: -0.298, Test Loss: 2183339.250, Test R²: -0.319
Epoch: 009, Train Loss: 2222829.500, Train R²: -0.288, Test Loss: 2166664.500, Test R²: -0.309
Epoch: 010, Train Loss: 2204700.500, Train R²: -0.277, Test Loss: 2149006.500, Test R²: -0.298
Epoch: 011, Train Loss: 2185315.750, Trai

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.metrics import r2_score
import torch_geometric.transforms as T
import numpy as np
import pandas as pd
from src.utils import set_seed

# 设置随机种子
SEED = 5
set_seed(SEED)

# 假定 featureDF, ppi_matrix, 和 all_sequence_outputsnew 已经定义好

# 转换为PyTorch Tensor
edge_index = torch.tensor(np.array([ppi_matrix.row, ppi_matrix.col]), dtype=torch.long)

# 创建PyTorch Geometric的Data对象
graph = Data(
    x=torch.tensor(merged_df['rNC2'].to_numpy(), dtype=torch.float32).unsqueeze(1),
    edge_index=edge_index,
    y=torch.tensor(merged_df['NC3'].to_numpy(), dtype=torch.float32).unsqueeze(1),
    seq=torch.tensor(all_sequence_outputsnew, dtype=torch.float32),
    pause=torch.tensor(merged_df['High_Pause_Counts'].to_numpy(), dtype=torch.float32).unsqueeze(1)
)

# 划分训练集、验证集和测试集#这个和上面划分效果差不多，测试过了
split = T.RandomNodeSplit(num_val=0.0, num_test=0.1)
graph = split(graph)

class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(1, 32)
        )

        self.encoder = nn.Sequential(
            nn.Linear(9216, 32),
            nn.GELU(),
        )

        self.conv1 = GCNConv(64, 1)
        self.fc1 = nn.Sequential(
            nn.Linear(64, 32),
            nn.GELU(),
            nn.Linear(32, 1)
        )

    def forward(self, data):
        x = data.x
        seq_embedding = data.seq
        edge_index = data.edge_index
        pausescore = data.pause

        x = self.fc(x) + self.encoder(seq_embedding)
        x = torch.cat((self.fc(pausescore), x), dim=1)
        x = F.gelu(x)
        #print(x.shape)
        #x = self.conv1(x, edge_index)
        x = self.fc1(x)
        #print(f'final output {x.shape}')

    
        return x

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
neural_net = NeuralNet().to(device)

# 损失函数和优化器
optimizer = optim.Adam(neural_net.parameters(), lr=0.005)
criterion = nn.MSELoss()

num_epochs = 1000000

graph = graph.to(device)

for epoch in range(1, num_epochs + 1):
    neural_net.train()
    optimizer.zero_grad()

    # 获取原始特征和掩码特征
    out = neural_net(graph)
    train_loss = criterion(out[graph.train_mask], graph.y[graph.train_mask])
    train_loss.backward()
    optimizer.step()

    if epoch %20 == 0:
        neural_net.eval()
        with torch.no_grad():
            out = neural_net(graph)
            train_loss = criterion(out[graph.train_mask], graph.y[graph.train_mask])
            train_r2 = r2_score(graph.y[graph.train_mask].cpu().numpy(), out[graph.train_mask].cpu().numpy())

            # val_loss = criterion(out[graph.val_mask], graph.y[graph.val_mask])
            # val_r2 = r2_score(graph.y[graph.val_mask].cpu().numpy(), out[graph.val_mask].cpu().numpy())

            test_loss = criterion(out[graph.test_mask], graph.y[graph.test_mask])
            test_r2 = r2_score(graph.y[graph.test_mask].cpu().numpy(), out[graph.test_mask].cpu().numpy())

        print(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.3f}, Train R²: {train_r2:.3f}, Test Loss: {test_loss:.3f}, Test R²: {test_r2:.3f}')


seed = 5
Epoch: 020, Train Loss: 1215026.375, Train R²: 0.287, Test Loss: 984010.500, Test R²: 0.342
Epoch: 040, Train Loss: 1099386.500, Train R²: 0.355, Test Loss: 865000.625, Test R²: 0.422
Epoch: 060, Train Loss: 1093466.750, Train R²: 0.359, Test Loss: 869298.438, Test R²: 0.419
Epoch: 080, Train Loss: 1089486.500, Train R²: 0.361, Test Loss: 869460.375, Test R²: 0.419
Epoch: 100, Train Loss: 1084752.000, Train R²: 0.364, Test Loss: 865629.438, Test R²: 0.421
Epoch: 120, Train Loss: 1078264.500, Train R²: 0.368, Test Loss: 864248.562, Test R²: 0.422
Epoch: 140, Train Loss: 1069508.000, Train R²: 0.373, Test Loss: 863702.688, Test R²: 0.423
Epoch: 160, Train Loss: 1058182.250, Train R²: 0.379, Test Loss: 864331.375, Test R²: 0.422
Epoch: 180, Train Loss: 1044473.375, Train R²: 0.387, Test Loss: 867390.125, Test R²: 0.420
Epoch: 200, Train Loss: 1028420.562, Train R²: 0.397, Test Loss: 873237.438, Test R²: 0.416
Epoch: 220, Train Loss: 1010177.500, Train R²: 0.408, Test Loss: 883538

KeyboardInterrupt: 

加上权重

In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.metrics import r2_score
import torch_geometric.transforms as T
import numpy as np
import pandas as pd
from src.utils import set_seed

# 设置随机种子
SEED = 5
set_seed(SEED)

# 假定 featureDF, ppi_matrix, 和 all_sequence_outputsnew 已经定义好

# 转换为PyTorch Tensor
edge_index = torch.tensor(np.array([ppi_matrix.row, ppi_matrix.col]), dtype=torch.long)
edge_weight = torch.tensor(ppi_matrix.data, dtype=torch.float32)

# 创建PyTorch Geometric的Data对象
graph = Data(
    x=torch.tensor(merged_df['rNC2'].to_numpy(), dtype=torch.float32).unsqueeze(1),
    edge_index=edge_index,
    edge_weight=edge_weight,
    y=torch.tensor(merged_df['NC3'].to_numpy(), dtype=torch.float32).unsqueeze(1),
    seq=torch.tensor(all_sequence_outputsnew, dtype=torch.float32),
    pause=torch.tensor(merged_df['High_Pause_Counts'].to_numpy(), dtype=torch.float32).unsqueeze(1)
)

# 划分训练集、验证集和测试集#这个和上面划分效果差不多，测试过了
split = T.RandomNodeSplit(num_val=0.0, num_test=0.1)
graph = split(graph)

class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(1, 32)
        )

        self.encoder = nn.Sequential(
            nn.Linear(9216, 32),
            nn.GELU(),
        )

        self.conv1 = GCNConv(64, 1)
        self.fc1 = nn.Sequential(
            nn.Linear(64, 32),
            nn.GELU(),
            nn.Linear(32, 1)
        )

    def forward(self, data):
        x = data.x
        seq_embedding = data.seq
        edge_index = data.edge_index
        edge_weight = data.edge_weight
        pausescore = data.pause

        x = self.fc(x) + self.encoder(seq_embedding)
        x = torch.cat((self.fc(pausescore), x), dim=1)
        x = F.gelu(x)
        x = self.conv1(x, edge_index, edge_weight=edge_weight)
        #x = self.fc1(x)

        return x

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
neural_net = NeuralNet().to(device)

# 损失函数和优化器
optimizer = optim.Adam(neural_net.parameters(), lr=0.005)
criterion = nn.MSELoss()

num_epochs = 1000000

graph = graph.to(device)

for epoch in range(1, num_epochs + 1):
    neural_net.train()
    optimizer.zero_grad()

    # 获取原始特征和掩码特征
    out = neural_net(graph)
    train_loss = criterion(out[graph.train_mask], graph.y[graph.train_mask])
    train_loss.backward()
    optimizer.step()

    if epoch % 20 == 0:
        neural_net.eval()
        with torch.no_grad():
            out = neural_net(graph)
            train_loss = criterion(out[graph.train_mask], graph.y[graph.train_mask])
            train_r2 = r2_score(graph.y[graph.train_mask].cpu().numpy(), out[graph.train_mask].cpu().numpy())

            # val_loss = criterion(out[graph.val_mask], graph.y[graph.val_mask])
            # val_r2 = r2_score(graph.y[graph.val_mask].cpu().numpy(), out[graph.val_mask].cpu().numpy())

            test_loss = criterion(out[graph.test_mask], graph.y[graph.test_mask])
            test_r2 = r2_score(graph.y[graph.test_mask].cpu().numpy(), out[graph.test_mask].cpu().numpy())

        print(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.3f}, Train R²: {train_r2:.3f}, Test Loss: {test_loss:.3f}, Test R²: {test_r2:.3f}')


seed = 5
Epoch: 020, Train Loss: 2063961.250, Train R²: -0.210, Test Loss: 1912968.125, Test R²: -0.279
Epoch: 040, Train Loss: 1963424.375, Train R²: -0.151, Test Loss: 1838486.625, Test R²: -0.229
Epoch: 060, Train Loss: 1915329.250, Train R²: -0.123, Test Loss: 1803173.500, Test R²: -0.205
Epoch: 080, Train Loss: 1878073.625, Train R²: -0.101, Test Loss: 1777382.500, Test R²: -0.188
Epoch: 100, Train Loss: 1847095.875, Train R²: -0.083, Test Loss: 1753783.125, Test R²: -0.172
Epoch: 120, Train Loss: 1825744.750, Train R²: -0.071, Test Loss: 1735472.625, Test R²: -0.160
Epoch: 140, Train Loss: 1813823.750, Train R²: -0.064, Test Loss: 1723657.375, Test R²: -0.152
Epoch: 160, Train Loss: 1805512.875, Train R²: -0.059, Test Loss: 1715047.750, Test R²: -0.146
Epoch: 180, Train Loss: 1799773.750, Train R²: -0.055, Test Loss: 1709228.375, Test R²: -0.143
Epoch: 200, Train Loss: 1795655.750, Train R²: -0.053, Test Loss: 1705703.000, Test R²: -0.140
Epoch: 220, Train Loss: 1792116.750, Trai

KeyboardInterrupt: 

In [24]:
unique_proteins = pd.unique(ppi[['protein_id2', 'protein_id1']].values.ravel('K'))
protein_id_map = {protein: idx for idx, protein in enumerate(unique_proteins)}

merged_df['ensp_id'] = merged_df['protein'].map(protein_id_map)
ppi['ensp_id2'] = ppi['protein_id2'].map(protein_id_map)
ppi['ensp_id1'] = ppi['protein_id1'].map(protein_id_map)
print(merged_df.head())
print(ppi.head())

   Unnamed: 0       transcript_id            protein  rNC1  rNC2     NC1  \
0           1  ENST00000000233.10  ENSP00000000233.5   449   379  1451.3   
1          18   ENST00000000412.8  ENSP00000000412.3   443   384   758.3   
2          37   ENST00000001008.6  ENSP00000001008.4  1015   899  5355.8   
3          62   ENST00000002125.9  ENSP00000002125.4   203   134   479.2   
4          87  ENST00000002165.11  ENSP00000002165.5   905   750   556.8   

      NC2     NC3         protein_id  High_Pause_Counts  ensp_id  
0  1458.2  1561.2  ENSP00000000233.5                7.0   2453.0  
1   817.2   815.6  ENSP00000000412.3                4.0   3604.0  
2  5303.3  5393.0  ENSP00000001008.4                9.0   3579.0  
3   562.1   563.5  ENSP00000002125.4               13.0    758.0  
4   567.1   605.8  ENSP00000002165.5               16.0   2185.0  
    Unnamed: 0        protein_id2        protein_id1  Protein1  Protein2  \
80          81  ENSP00000317992.6  ENSP00000219240.4     209.0   

In [25]:
#ppi直接加载构建好的npz，以下是构建npz步骤

#用ppi构建一个coo的稀疏举证，用protein1和protein2构建一个coo的稀疏矩阵
ppi_matrix = sp.coo_matrix((ppi["CombinedScore"], (ppi['ensp_id2'], ppi['ensp_id1'])), shape=(merged_df.shape[0], merged_df.shape[0]))

ppi_matrix

<7132x7132 sparse matrix of type '<class 'numpy.int64'>'
	with 364726 stored elements in COOrdinate format>

In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.metrics import r2_score
import torch_geometric.transforms as T
import numpy as np
import pandas as pd
from src.utils import set_seed

# 设置随机种子
SEED = 5
set_seed(SEED)

# 假定 featureDF, ppi_matrix, 和 all_sequence_outputsnew 已经定义好

# 转换为PyTorch Tensor
edge_index = torch.tensor(np.array([ppi_matrix.row, ppi_matrix.col]), dtype=torch.long)

# 创建PyTorch Geometric的Data对象
graph = Data(
    x=torch.tensor(merged_df['rNC2'].to_numpy(), dtype=torch.float32).unsqueeze(1),
    edge_index=edge_index,
    y=torch.tensor(merged_df['NC3'].to_numpy(), dtype=torch.float32).unsqueeze(1),
    seq=torch.tensor(all_sequence_outputsnew, dtype=torch.float32),
    pause=torch.tensor(merged_df['High_Pause_Counts'].to_numpy(), dtype=torch.float32).unsqueeze(1)
)

# 划分训练集、验证集和测试集#这个和上面划分效果差不多，测试过了
split = T.RandomNodeSplit(num_val=0.0, num_test=0.1)
graph = split(graph)

class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(1, 32)
        )

        self.encoder = nn.Sequential(
            nn.Linear(9216, 32),
            nn.GELU(),
        )

        self.conv1 = GCNConv(64, 1)
        self.fc1 = nn.Sequential(
            nn.Linear(64, 32),
            nn.GELU(),
            nn.Linear(32, 1)
        )

    def forward(self, data):
        x = data.x
        seq_embedding = data.seq
        edge_index = data.edge_index
        pausescore = data.pause

        x = self.fc(x) + self.encoder(seq_embedding)
        x = torch.cat((self.fc(pausescore), x), dim=1)
        x = F.gelu(x)
        #print(x.shape)
        #x = self.conv1(x, edge_index)
        x = self.fc1(x)
        #print(f'final output {x.shape}')

    
        return x

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
neural_net = NeuralNet().to(device)

# 损失函数和优化器
optimizer = optim.Adam(neural_net.parameters(), lr=0.005)
criterion = nn.MSELoss()

num_epochs = 1000000

graph = graph.to(device)

for epoch in range(1, num_epochs + 1):
    neural_net.train()
    optimizer.zero_grad()

    # 获取原始特征和掩码特征
    out = neural_net(graph)
    train_loss = criterion(out[graph.train_mask], graph.y[graph.train_mask])
    train_loss.backward()
    optimizer.step()

    if epoch %20 == 0:
        neural_net.eval()
        with torch.no_grad():
            out = neural_net(graph)
            train_loss = criterion(out[graph.train_mask], graph.y[graph.train_mask])
            train_r2 = r2_score(graph.y[graph.train_mask].cpu().numpy(), out[graph.train_mask].cpu().numpy())

            # val_loss = criterion(out[graph.val_mask], graph.y[graph.val_mask])
            # val_r2 = r2_score(graph.y[graph.val_mask].cpu().numpy(), out[graph.val_mask].cpu().numpy())

            test_loss = criterion(out[graph.test_mask], graph.y[graph.test_mask])
            test_r2 = r2_score(graph.y[graph.test_mask].cpu().numpy(), out[graph.test_mask].cpu().numpy())

        print(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.3f}, Train R²: {train_r2:.3f}, Test Loss: {test_loss:.3f}, Test R²: {test_r2:.3f}')


seed = 5
Epoch: 020, Train Loss: 1215026.375, Train R²: 0.287, Test Loss: 984010.500, Test R²: 0.342
Epoch: 040, Train Loss: 1099386.500, Train R²: 0.355, Test Loss: 865000.625, Test R²: 0.422
Epoch: 060, Train Loss: 1093466.750, Train R²: 0.359, Test Loss: 869298.438, Test R²: 0.419
Epoch: 080, Train Loss: 1089486.500, Train R²: 0.361, Test Loss: 869460.375, Test R²: 0.419
Epoch: 100, Train Loss: 1084752.000, Train R²: 0.364, Test Loss: 865629.438, Test R²: 0.421
Epoch: 120, Train Loss: 1078264.500, Train R²: 0.368, Test Loss: 864248.562, Test R²: 0.422
Epoch: 140, Train Loss: 1069508.000, Train R²: 0.373, Test Loss: 863702.688, Test R²: 0.423
Epoch: 160, Train Loss: 1058182.250, Train R²: 0.379, Test Loss: 864331.375, Test R²: 0.422
Epoch: 180, Train Loss: 1044473.375, Train R²: 0.387, Test Loss: 867390.125, Test R²: 0.420
Epoch: 200, Train Loss: 1028420.562, Train R²: 0.397, Test Loss: 873237.438, Test R²: 0.416
Epoch: 220, Train Loss: 1010177.500, Train R²: 0.408, Test Loss: 883538

KeyboardInterrupt: 